In [1]:
from __future__ import absolute_import, division, print_function
import os

PREF = '002a02' 
KAGGLE = 'KAGGLE_WORKING_DIR' in os.environ
COLAB = 'COLAB_GPU' in os.environ

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
from multiprocessing import Pool, cpu_count
import tensorflow as tf
from keras import backend as K
import keras
from PIL import Image

import time

kernel_time = time.time()

def tic_tac(restart):
    global kernel_time 
    
    if restart:
        kernel_time = time.time()
    else:
        t_sec = round(time.time() - kernel_time)
        (t_min, t_sec) = divmod(t_sec,60)
        (t_hour,t_min) = divmod(t_min,60) 
        print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))
    return

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
    tf.set_random_seed(seed)

tic_tac(True)
PREF

Using TensorFlow backend.


'002a02'

In [22]:
#import keras
#from keras import backend as K

import tensorflow
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
#%matplotlib inline


In [3]:
import tensorflow as tf
import keras.backend as K


random_state = 21
seed_everything(random_state)

if KAGGLE:
    OUTPUT_DIR = './'
    TRAIN_DIR = '../input/skin-cancer-mnist-ham10000/' # directory of training images
    PRETRAINED_MODEL_PATH = '{}__weights.h5'.format(PREF)   
    IMAGES_DIR = '../input/skin-cancer-mnist-ham10000/ham10000_images_part_1/'
elif COLAB:
    OUTPUT_DIR = './gdrive/My Drive/skin-cancer-mnist-ham10000/output/'
    TRAIN_DIR = './gdrive/My Drive/skin-cancer-mnist-ham10000/input/' # directory of training images
    PRETRAINED_MODEL_PATH = '{}__weights.h5'.format(PREF)   
    IMAGES_DIR = './letter_images/'
else:
    OUTPUT_DIR = '../output/'
    TRAIN_DIR = '../input/' # directory of training images
    PRETRAINED_MODEL_PATH = '{}__weights.h5'.format(PREF)   
    IMAGES_DIR = TRAIN_DIR

num_cores = cpu_count()

if KAGGLE:
    GPU = True; CPU = False
else:
    GPU = False; CPU = True
if GPU: num_GPU = 1;num_CPU = 1
if CPU: num_CPU = 1; num_GPU = 0

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores, 
                        gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.25),
                        allow_soft_placement=True, device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})    

_ = tf.Session(config=config)

def set_session():
    _ = tf.Session(config=config)
    sess = tf.Session(config=config)
    K.tensorflow_backend.set_session(sess)
    K.set_session(sess)
set_session()

N_SPLITS=5

# Kernel Configurations
LOAD_PRETRAINED_MODEL = True # load a pre-trained model
SAVE_MODEL = True # save the model after training
TRAIN_IMAGE_DIR = os.path.join(TRAIN_DIR, 'train_images') # 

BATCH_SIZE = 32
EPOCHS=1

REDUCE_LOAD=False
N_LOOPS = 12

# Constant
kernel_start = kernel_time #time.time()
kernel_time_limit = 60*60*6.5       #### running time

IMG_W = 300 # resized weidth
IMG_H = 224 # resized heigh

# Input image dimensions.
INPUT_SHAPE = (IMG_H, IMG_W, 3) #x_train.shape[1:]


In [4]:
import os
import logging
import time
from datetime import timedelta
import pandas as pd

class LogFormatter():

    def __init__(self):
        self.start_time = time.time()

    def format(self, record):
        elapsed_seconds = round(record.created - self.start_time)

        prefix = "%s - %s - %s" % (
            record.levelname,
            time.strftime('%x %X'),
            timedelta(seconds=elapsed_seconds)
        )
        message = record.getMessage()
        message = message.replace('\n', '\n' + ' ' * (len(prefix) + 3))
        return "%s - %s" % (prefix, message) if message else ''

def create_logger(filepath, rank):
    """
    Create a logger.
    Use a different log file for each process.
    """
    # create log formatter
    log_formatter = LogFormatter()

    # create file handler and set level to debug
    if filepath is not None:
        if rank > 0:
            filepath = '%s-%i' % (filepath, rank)
        file_handler = logging.FileHandler(filepath, "a")
        file_handler.setLevel(logging.INFO)
        file_handler.setFormatter(log_formatter)

    # create console handler and set level to info
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(log_formatter)

    # create logger and set level to debug
    logger = logging.getLogger()
    logger.handlers = []
    logger.setLevel(logging.INFO)
    logger.propagate = False
    if filepath is not None:
        logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    # reset logger elapsed time
    def reset_time():
        log_formatter.start_time = time.time()
    logger.reset_time = reset_time

    return logger


logger = create_logger(os.path.join(OUTPUT_DIR, '{}__train.log'.format(PREF)), rank=0)
logger.info("============ Initialized logger ============")
logger.info("")

INFO - 10/04/19 08:48:04 - 0:00:00 - ============ Initialized logger ============



In [5]:
# Resnet definition.
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = False
num_classes = 8

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = False

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

def lr_schedule(epoch):
    global logger
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 2.5e-4
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    logger.info('Learning rate: {}'.format(lr))
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10, include_top=False):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    if include_top:
        x = AveragePooling2D(pool_size=8)(x)
        y = Flatten()(x)
        outputs = Dense(num_classes,
                        activation='softmax',
                        kernel_initializer='he_normal')(y)
    else:
        x = AveragePooling2D(pool_size=4)(x)
        y = Flatten()(x)
        outputs = y

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model



In [6]:
# Create Train and Val Sets
df_data = pd.read_csv(TRAIN_DIR+'HAM10000_metadata.csv')
#df_data.head()

In [7]:
labels = np.unique(df_data.dx)

df_data['label'] = 0
for num, label in enumerate(labels):
    df_data.loc[df_data.dx == label, 'label'] = num

## Note: the relation lesion_id to image_id is one to many.

# train_test and val split
df_data_hat = df_data.loc[:,['lesion_id','dx']].drop_duplicates()
df_data_hat['dx'].value_counts()

nv       5403
bkl       727
mel       614
bcc       327
akiec     228
vasc       98
df         73
Name: dx, dtype: int64

In [8]:
df_data_hat = df_data_hat.loc[:,['lesion_id']].merge(df_data, how='left', on='lesion_id')
#df_data_hat.head()

In [9]:
# https://www.kaggle.com/ateplyuk/pytorch-starter-u-net-resnet
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

class DataGenerator(keras.utils.Sequence):
    def __init__(self, df, y=None, batch_size = 16, subset="train", shuffle=False, 
                preprocess=None, augmentation=None, 
                info={}):
        super().__init__()
        self.df = df
        self.y = y
        self.shuffle = shuffle
        self.subset = subset
        self.batch_size = batch_size
        self.preprocess = preprocess
        self.augmentation = augmentation
        self.info = info
        self.on_epoch_end()
        self.data_path = IMAGES_DIR

    def __len__(self):
        if len(self.df) // self.batch_size == 0:
            return int(np.floor(len(self.df) / self.batch_size))
        else:
            return int(np.floor(len(self.df) / self.batch_size))+1
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    

    def __getitem__(self, index): 
        X = np.empty((self.batch_size,IMG_H,IMG_W,3),dtype=np.float32)
#         y = np.zeros((self.batch_size, len(labels)),dtype=np.int8)
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        tmp = self.df.loc[:,['image_id','label']].iloc[indexes]
        if self.subset == 'train': 
            y = self.y[indexes]
        for i in range(tmp.shape[0]):
            self.info[index*self.batch_size+i]=tmp.iloc[i,0]
            X[i] = Image.open(self.data_path+tmp.iloc[i,0]+'.jpg').resize((IMG_W,IMG_H))
        # apply augmentations
        if self.augmentation:
            for n in range(X.shape[0]):
                X[n] = self.augmentation()(image=X[n].astype('uint8'))['image']
        
        if self.preprocess!=None:
            for func in self.preprocess:
                X = func(X)

        if self.subset == 'train': 
            return X[:tmp.shape[0]], y[:tmp.shape[0]]
        else: 
            return X[:tmp.shape[0]], None

In [10]:
# Augmentation
# import cv2
# from albumentations import (
#     Compose, HorizontalFlip, CLAHE, HueSaturationValue,
#     RandomBrightness, RandomContrast, RandomGamma,OneOf,
#     ToFloat, ShiftScaleRotate,GridDistortion, ElasticTransform, JpegCompression, HueSaturationValue,
#     RGBShift, RandomBrightness, RandomContrast, Blur, MotionBlur, MedianBlur, GaussNoise,CenterCrop,
#     IAAAdditiveGaussianNoise,GaussNoise,OpticalDistortion,RandomSizedCrop
# )

import albumentations as A

def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)

# define heavy augmentations
def get_training_augmentation():
    train_transform = [

        A.HorizontalFlip(p=0.5),

        A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),
        A.IAAAdditiveGaussianNoise(p=0.2),
        A.IAAPerspective(p=0.5),

        A.OneOf(
            [
                A.CLAHE(p=1),
                A.RandomBrightness(p=1),
                A.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.IAASharpen(p=1),
                A.Blur(blur_limit=3, p=1),
                A.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        A.OneOf(
            [
                A.RandomContrast(p=1),
                A.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
        #A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform, p=0.6)




In [11]:
def adjustData(img):
    if(np.max(img) > 1):
        img = img.astype(np.float32) / 255.0
    return img


preprocess_input = adjustData

# # Normalize data.
# x_train = x_train.astype('float32') / 255
# x_test = x_test.astype('float32') / 255

# # If subtract pixel mean is enabled
# if subtract_pixel_mean:
#     x_train_mean = np.mean(x_train, axis=0)
#     x_train -= x_train_mean
#     x_test -= x_train_mean


In [12]:
# Create train/validation split stratified by no conformity
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

if REDUCE_LOAD:
    df_data_hat = df_data_hat.iloc[:int(0.12*df_data_hat.shape[0]),:]
    

all_batches = DataGenerator(df_data_hat,shuffle = False, preprocess=[adjustData], subset='test',
        batch_size=BATCH_SIZE)


In [13]:
# plots images with labels within jupyter notebook
# source: https://github.com/smileservices/keras_utils/blob/master/utils.py

def plots(ims, figsize=(16,6), rows=4, interp=False, titles=None): # 12,6
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

#plots((all_batches[0][0])*255.0, titles=all_batches[0][1]) # titles=labels will display the image labels

In [14]:
# Predict all dataset in backbone
def get_model(include_top=True, n_dim=1, weights_file=None):
    multiple = keras.applications.vgg16.VGG16(include_top=False, input_shape=INPUT_SHAPE, weights='imagenet', pooling=None)
    inputs = multiple.layers[0].input
    x = multiple.layers[-1].output
    y = Flatten()(x)   
#     inputs = Input(shape=INPUT_SHAPE)
#     x = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', pooling=None)(inputs)
#     y = Flatten()(x)
    if include_top:
        outputs = Dense(units=n_dim,
                        activation='softmax',
                        kernel_initializer='he_normal')(y)
        model = Model(inputs=inputs, outputs=outputs)
        if weights_file is not None:
            model.load_weights(weights_file)
            return model
        else:
            model_hat = get_model(include_top=False, weights_file=None)
            logger.info('    knowledge_transfer ready!.')
            return knowledge_transfer(model_hat, model)
    else:
        outputs = y
        model = Model(inputs=inputs, outputs=outputs)
        if weights_file is None:
            return model
        else:
            model_hat = get_model(include_top=False, weights_file=PRETRAINED_MODEL_PATH)
            logger.info('    knowledge_transfer ready!.')
            return knowledge_transfer(model_hat, model)
            
    return None


def knowledge_transfer(teacher, student, weights_file=None):
    for n in range(min(len(teacher.layers),len(student.layers))):
        student.layers[n].set_weights(teacher.layers[n].get_weights())
    
    return student

def generate_features(datagen, n_dim=None, weights_file=None):
    logger.info('0) generating features from dataset. (aprox. 700 s)')
    
    model = get_model(include_top=False, n_dim=n_dim, weights_file=weights_file)
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr_schedule(0)),
                  metrics=['accuracy'])
    
    
    y_pred = model.predict_generator(datagen,verbose=0)
    
    logger.info('    done.')
    return y_pred



In [15]:
# PCA transformation
from sklearn.decomposition import PCA
import pickle
import time


def dim_reduction(y_pred):
    logger.info('1) dim reduction by PCA.')

    pca = PCA(n_components=256, random_state=random_state)
    pca = pca.fit(y_pred)
    y_pca = pca.transform(y_pred)


    timestr = time.strftime("%Y%m%d-%H%M%S")
    pickle_out = open("{}{}__PCA_transform_{}.pickle".format(OUTPUT_DIR, PREF, timestr),"wb")
    pickle.dump(pca, pickle_out)
    pickle_out.close()
    
    logger.info('    done.')
    return y_pca


In [16]:
# HDScan clustering
if KAGGLE or COLAB:
    !pip install hdbscan
    
import hdbscan

def clustering(y_pca, min_cluster_size=60):
    logger.info('2) Clustering (HDBScan).')

    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=4).fit(y_pca) 

    y_cluster = clusterer.labels_ # clusterer.probabilities_

    labels, labels_ct = np.unique(y_cluster, return_counts=True)
    for label, ct in zip(labels, labels_ct):
        logger.info('        label: {}, count: {} ({}%).'.format(label, ct, 100.0* ct/np.sum(labels_ct)))
    # test_labels, strengths = hdbscan.approximate_predict(clusterer, test_points)
    # test_labels
    timestr = time.strftime("%Y%m%d-%H%M%S")
    pickle_out = open("{}{}__HDBScan_{}.pickle".format(OUTPUT_DIR, PREF, timestr),"wb")
    pickle.dump(clusterer, pickle_out)
    pickle_out.close()
    
    logger.info('    done.')

    y_cluster_encoded = np.zeros((y_cluster.shape[0], labels.shape[0]))
    for n in range(y_cluster.shape[0]):
        y_cluster_encoded[n, list(labels).index(y_cluster[n])] = 1
        
    logger.info('    n_clusters={}'.format(y_cluster_encoded.shape[1]))
    
    return y_cluster, y_cluster_encoded, y_cluster_encoded.shape[1]

In [17]:
# # Create train/validation split stratified on y_cluster
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import StratifiedKFold

# train_test_df_data_hat = train_test_df_data.loc[:,['lesion_id','dx']].drop_duplicates()

# folds = list(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=random_state).split(df_data_hat, y_cluster))


# n_split = 0 # for next
# current_model='Resnet20'
# df_tr = df_data_hat.iloc[folds[n_split][0]]
# df_ts = df_data_hat.iloc[folds[n_split][1]]

# train_batches = DataGenerator(df_tr,shuffle = True, preprocess=[adjustData], batch_size=BATCH_SIZE)
# valid_batches = DataGenerator(df_ts,preprocess=[adjustData], batch_size=BATCH_SIZE)



In [18]:
# Define Top2 and Top3 Accuracy

def top_3_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=2)

In [19]:
# Keras model
from keras.models import Sequential, Model
from keras.models import model_from_json
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization, GaussianNoise
from keras import callbacks
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score


# LOGGER
class Logger(callbacks.Callback):
    def __init__(self, out_path='./', patience=10, lr_patience=3, out_fn='', log_fn='', verbose=True, 
                 Xval=None, yval=None, val_gen=None):
        self.score = -1000
        self.scores = []
        self.path = out_path
        self.fn = out_fn
        self.patience = patience
        self.lr_patience = lr_patience
        self.no_improve = 0
        self.no_improve_lr = 0
        self.Xval = Xval
        self.yval = yval
        self.val_gen = val_gen
        self.verbose = verbose

    def on_train_begin(self, logs={}):
        if self.val_gen is not None:
            y_true = None
            for n in range(len(self.val_gen)):
                if y_true is None:
                    y_true = np.argmax( self.val_gen[n][1], axis=1).reshape((-1,1))
                else:
                    y_true = np.vstack((y_true,np.argmax(self.val_gen[n][1], axis=1).reshape((-1,1)) ))
            self.yval= y_true.reshape((-1,1))
        
        #self.on_epoch_end(-1, logs=logs) ##################################################
        
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        cv_pred = self.model.predict_generator(self.val_gen)

        y_pred = np.zeros((len(cv_pred),1))
        for n in range(len(cv_pred)):
            y_pred[n] = np.argmax(cv_pred[n])
        cv_true = self.yval
        score = f1_score(cv_true, y_pred, average='macro')
        self.scores.append(score)
        if self.score < score:
            if epoch == -1: print('Initial F1 score')
            self.no_improve = 0
            self.no_improve_lr = 0
            if self.verbose:
                print("Epoch %s - best F1 score: %s" % (epoch+1, round(score, 5)))
            self.score = score
            #self.model.save_weights(self.path + self.fn, overwrite=True)
            save_weights(self.model, self.path + self.fn)
        else:
            self.no_improve += 1
            self.no_improve_lr += 1
            if self.verbose:
                print("Epoch %s - current F1 score: %s - best F1 score: %s" % (epoch+1, round(score, 5), round(self.score, 5)))
            if self.no_improve >= self.patience:
                if self.verbose:
                    print("Epoch %s - current F1 score: %s - best F1 score: %s" % (epoch+1, round(score, 5), round(self.score, 5)))
                    print('Trained stoped.')
                else:
                    print("Epoch %s - current F1 score: %s - best F1 score: %s" % (epoch+1, round(score, 5), round(self.score, 5)))
                self.model.stop_training = True
            if self.no_improve_lr >= self.lr_patience:
                lr = float(K.get_value(self.model.optimizer.lr))
                lr *= 10
                K.set_value(self.model.optimizer.lr, lr)
                if self.verbose:
                    print("Setting lr to {}".format(lr))
                self.no_improve_lr = 0
            else:
                lr = float(K.get_value(self.model.optimizer.lr))
                lr *= 0.975
                K.set_value(self.model.optimizer.lr, lr)
                if self.verbose:
                    print("Setting lr to {}".format(lr))
    
        if time.time() - kernel_start > kernel_time_limit:
            self.model.stop_training = True
            print('Time Limit reached.')
        return

    # matthews_corrcoef


In [20]:
def save_weights(model, file_name):
    states_h = []
    states_l = []
    for l in model.layers:
        states_h.append(l.trainable)
        l.trainable = True
#     for l in model.layers[1].layers:
#         states_l.append(l.trainable)
#         l.trainable = True

    model.save_weights(file_name, overwrite=True)

    for n, l in enumerate(model.layers):
        l.trainable = states_h[n]
#     for n, l in enumerate(model.layers[1].layers):
#         l.trainable = states_l[n]

    return
    
def load_weights(model, weights_model_fn = PRETRAINED_MODEL_PATH):
    states_h = []
    states_l = []
    for l in model.layers:
        states_h.append(l.trainable)
        l.trainable = True
#     for l in model.layers[1].layers:
#         states_l.append(l.trainable)
#         l.trainable = True

    model.load_weights(weights_model_fn, by_name=True)


    for n, l in enumerate(model.layers):
        l.trainable = states_h[n]
#     for n, l in enumerate(model.layers[1].layers):
#         l.trainable = states_l[n]
    return model


In [21]:
import pickle
from sklearn.model_selection import StratifiedKFold

n_split = 0
# for n_split in range(N_SPLITS):
def training_on_pseudolabels(df_data_hat, y_cluster_encoded, y_cluster, epochs=2):
    global logger
    logger.info('3) pseudolabeling training.')

    K.clear_session()
    _ = tf.Session(config=config)

    sess = tf.Session(config=config)
    K.tensorflow_backend.set_session(sess)
    K.set_session(sess)

    keras.backend.set_image_data_format('channels_last')

    current_model='Resnet20'
    
    folds = list(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=random_state).split(df_data_hat, y_cluster))

    df_tr = df_data_hat.loc[folds[n_split][0]]
    ytr = y_cluster_encoded[folds[n_split][0]]
    df_ts = df_data_hat.loc[folds[n_split][1]]
    yts = y_cluster_encoded[folds[n_split][1]]

    train_batches = DataGenerator(df_tr, y=ytr, shuffle = True, preprocess=[adjustData], batch_size=BATCH_SIZE)
    valid_batches = DataGenerator(df_ts, y=yts, preprocess=[adjustData], batch_size=BATCH_SIZE)

    klogger = Logger(patience=60, lr_patience=16, out_path='./', val_gen=valid_batches,
                   out_fn=PRETRAINED_MODEL_PATH, verbose=True)

    weights_file = None
    if os.path.isfile(PRETRAINED_MODEL_PATH):
        weights_file = PRETRAINED_MODEL_PATH
    model = get_model(include_top=True, n_dim=ytr.shape[1], weights_file=weights_file)

    model.compile(optimizer=Adam(lr_schedule(200)), loss='categorical_crossentropy', 
          metrics=[keras.metrics.categorical_accuracy, top_2_accuracy, top_3_accuracy])

    # Add weights to try to make the model more sensitive to melanoma
    history = model.fit_generator(train_batches,  
                                  #class_weight=class_weights,
                        validation_data=valid_batches,
                        epochs=epochs, verbose=1,
                       callbacks=[klogger])

    timestr = time.strftime("%Y%m%d-%H%M%S")
    model_weihgts_fn = PRETRAINED_MODEL_PATH[:-3]+timestr+'.h5'

    cmd = 'cp {} {}'.format(PRETRAINED_MODEL_PATH,model_weihgts_fn)
    os.system(cmd)

    with open('{}_{}_scores_{}.pickle'.format(PREF, current_model, n_split), 'wb') as handle:
        pickle.dump(klogger.scores, handle)
    with open('{}_{}_history_{}.pickle'.format(PREF, current_model, n_split), 'wb') as handle:
        pickle.dump(history, handle)

    # Score trained model.
    model = load_weights(model,PRETRAINED_MODEL_PATH)


    scores = model.evaluate_generator(valid_batches, verbose=1)
    logger.info('    Val loss:{}'.format(scores[0]))
    logger.info('    Val accuracy:{}'.format(scores[1]))
    logger.info('  done.')
    
    return







In [ ]:
## Main loop
n_dim = 3
for n in range(1,N_LOOPS):
    if n > 0:
        y_pred = generate_features(all_batches, n_dim, PRETRAINED_MODEL_PATH) # Aqui para resumir snapshot
    else:
        y_pred =  generate_features(all_batches)
    y_pca = dim_reduction(y_pred)
#     y_cluster, y_cluster_encoded, n_dim = clustering(y_pca)
    min_cluster_size = 60
    while n_dim == 1:
        y_cluster, y_cluster_encoded, n_dim = clustering(y_pca, min_cluster_size=min_cluster_size)
        min_cluster_size -= 2
        logger.warning('    n_dim = 1 in clustering output.')
        
    training_on_pseudolabels(df_data_hat, y_cluster_encoded, y_cluster, epochs=2**n)
    if time.time() - kernel_start > kernel_time_limit:
        print('Time Limit reached.')
        break


INFO - 10/04/19 08:36:43 - 0:00:02 - 0) generating features from dataset. (aprox. 700 s)
WARNING - 10/04/19 08:36:43 - 0:00:02 - From /home/maxim/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.
                                        
WARNING - 10/04/19 08:36:43 - 0:00:02 - From /home/maxim/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.
                                        
WARNING - 10/04/19 08:36:43 - 0:00:02 - From /home/maxim/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.
                                        
WARNING - 10/04/19 08:36:43 - 0:00:02 - From /home/maxim/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3

In [ ]:
# clustering_score: 

In [ ]:
tic_tac(False)

In [ ]:
# EOF

In [ ]:

# if version == 2:
#     model = resnet_v2(input_shape=input_shape, depth=depth)
# else:
#     model = resnet_v1(input_shape=input_shape, depth=depth)

# model.compile(loss='categorical_crossentropy',
#               optimizer=Adam(learning_rate=lr_schedule(0)),
#               metrics=['accuracy'])
# model.summary()
# print(model_type)

# # Prepare model model saving directory.
# save_dir = os.path.join(os.getcwd(), 'saved_models')
# model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# filepath = os.path.join(save_dir, model_name)

# # Prepare callbacks for model saving and for learning rate adjustment.
# checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_acc',
#                              verbose=1,
#                              save_best_only=True)

# lr_scheduler = LearningRateScheduler(lr_schedule)

# lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                cooldown=0,
#                                patience=5,
#                                min_lr=0.5e-6)

# callbacks = [checkpoint, lr_reducer, lr_scheduler]

# # Run training, with or without data augmentation.
# if not data_augmentation:
#     print('Not using data augmentation.')
#     model.fit(x_train, y_train,
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_data=(x_test, y_test),
#               shuffle=True,
#               callbacks=callbacks)
# else:
#     print('Using real-time data augmentation.')
#     # This will do preprocessing and realtime data augmentation:
#     datagen = ImageDataGenerator(
#         # set input mean to 0 over the dataset
#         featurewise_center=False,
#         # set each sample mean to 0
#         samplewise_center=False,
#         # divide inputs by std of dataset
#         featurewise_std_normalization=False,
#         # divide each input by its std
#         samplewise_std_normalization=False,
#         # apply ZCA whitening
#         zca_whitening=False,
#         # epsilon for ZCA whitening
#         zca_epsilon=1e-06,
#         # randomly rotate images in the range (deg 0 to 180)
#         rotation_range=0,
#         # randomly shift images horizontally
#         width_shift_range=0.1,
#         # randomly shift images vertically
#         height_shift_range=0.1,
#         # set range for random shear
#         shear_range=0.,
#         # set range for random zoom
#         zoom_range=0.,
#         # set range for random channel shifts
#         channel_shift_range=0.,
#         # set mode for filling points outside the input boundaries
#         fill_mode='nearest',
#         # value used for fill_mode = "constant"
#         cval=0.,
#         # randomly flip images
#         horizontal_flip=True,
#         # randomly flip images
#         vertical_flip=False,
#         # set rescaling factor (applied before any other transformation)
#         rescale=None,
#         # set function that will be applied on each input
#         preprocessing_function=None,
#         # image data format, either "channels_first" or "channels_last"
#         data_format=None,
#         # fraction of images reserved for validation (strictly between 0 and 1)
#         validation_split=0.0)

#     # Compute quantities required for featurewise normalization
#     # (std, mean, and principal components if ZCA whitening is applied).
#     datagen.fit(x_train)

#     # Fit the model on the batches generated by datagen.flow().
#     model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
#                         validation_data=(x_test, y_test),
#                         epochs=epochs, verbose=1, workers=4,
#                         callbacks=callbacks)

# # Score trained model.
# scores = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', scores[0])
# print('Test accuracy:', scores[1])